****Preprocessing

In [1]:
import torch
import torchvision
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
#*** PREPROCESSING ***

def preprocess(train_path, test_path, test=False):
  df = pd.read_csv(train_path)[:55000]
  df_test = pd.read_csv(test_path)
  
  df_test['Sales'] = 0
  df = pd.concat([df, df_test])

  df['month'] = pd.DatetimeIndex(df['Date']).month
  df['DayInMonth'] = pd.DatetimeIndex(df['Date']).day
  df = pd.get_dummies(data=df, columns=['Store','AssortmentType', 'StoreType', 'StateHoliday', "month", "DayOfWeek"])
  del df["Date"]

  df = df.dropna(subset=['Sales'])
  Y = df["Sales"].values
  del df["Sales"]

  df = df.values
    
  return df, Y

X_train, Y_train = preprocess("/content/drive/MyDrive/Lithuanian_Challenge/train_data.csv", "/content/drive/MyDrive/Lithuanian_Challenge/test_data.csv")

X_test_submission, X_train, Y_train = X_train[-41088:], X_train[:-41088], Y_train[:-41088]

X_test, Y_test = X_train[-50000:], Y_train[-50000:] 
X_train, Y_train = X_train[:-50000], Y_train[:-50000]  

print('Submission Set Shape: ',X_test_submission.shape)
print('Train Set Shape: ',X_train.shape)
print('Test Set Shape: ',X_test.shape)




/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Submission Set Shape:  (41088, 1140)
Train Set Shape:  (5000, 1140)
Test Set Shape:  (50000, 1140)


****CNN

In [8]:
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten, Dropout


from keras import backend as K
def rmse(y_true, y_pred):
        return K.sqrt(K.mean(K.square(float(y_pred) - float(y_true)))) 

X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

epochs = 3
batch = 256
lr = 0.001
adam = optimizers.Adam(lr)

model_cnn = Sequential()
model_cnn.add(Conv1D(filters=256, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Flatten())
model_cnn.add(Dropout(0.1))
model_cnn.add(Dense(1024, activation='relu'))
model_cnn.add(Dense(1))
model_cnn.compile(loss=rmse, optimizer=adam)
model_cnn.summary()


cnn_history = model_cnn.fit(X_train, Y_train, epochs=epochs, validation_data=(X_test, Y_test), verbose=1) 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 1139, 256)         768       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 569, 256)          0         
_________________________________________________________________
flatten (Flatten)            (None, 145664)            0         
_________________________________________________________________
dropout (Dropout)            (None, 145664)            0         
_________________________________________________________________
dense (Dense)                (None, 1024)              149160960 
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1025      
Total params: 149,162,753
Trainable params: 149,162,753
Non-trainable params: 0
________________________________________

In [12]:
model_cnn.predict(X_test_submission.reshape((X_test_submission.shape[0], X_test_submission.shape[1], 1)))

array([[233.794],
       [233.794],
       [233.794],
       ...,
       [233.794],
       [233.794],
       [233.794]], dtype=float32)